In [2]:
import numpy as np
import os
import sys
import pandas as pd

import sklearn as sk
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

import pylab as pl
import h5py

import tensorflow as tf
from tensorflow.keras import mixed_precision

os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
os.environ['CUDA_VISIBLE_DEVICES']="0" 

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)


from sklearn.metrics import confusion_matrix

base_dir = '/media/tord/T7/Thesis_ssd/MasterThesis3.0'
os.chdir(base_dir)
from Classes.DataProcessing.LoadData import LoadData
from Classes.DataProcessing.HelperFunctions import HelperFunctions
from Classes.DataProcessing.DataHandler import DataHandler
from Classes.DataProcessing.TimeAugmentor import TimeAugmentor
from Classes.DataProcessing.NoiseAugmentor import NoiseAugmentor
from Classes.DataProcessing.RamLoader import RamLoader
from Classes.DataProcessing.RamGenerator import RamGenerator
from Classes.Modeling.DynamicModels import DynamicModels
from Classes.Modeling.StaticModels import StaticModels
from Classes.Modeling.InceptionTimeModel import InceptionTimeModel
from Classes.Modeling.NarrowSearchRam import NarrowSearchRam
from Classes.Modeling.CustomCallback import CustomCallback
from Classes.Modeling.ResultFitter import ResultFitter
from Classes.Scaling.ScalerFitter import ScalerFitter
from Classes.Scaling.MinMaxScalerFitter import MinMaxScalerFitter
from Classes.Scaling.StandardScalerFitter import StandardScalerFitter
import json
#from Classes import Tf_shutup
#Tf_shutup.Tf_shutup()

from livelossplot import PlotLossesKeras



from matplotlib.colors import ListedColormap

plt.rcParams["figure.figsize"]= (15,15)
helper = HelperFunctions()

import sys
ISCOLAB = 'google.colab' in sys.modules

import random
import pprint

base_dir = '/media/tord/T7/Thesis_ssd/MasterThesis3.0'
os.chdir(base_dir)

mixed_precision.set_global_policy('mixed_float16')


%load_ext tensorboard

1 Physical GPUs, 1 Logical GPUs
INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: GeForce RTX 3090, compute capability 8.6


In [3]:
load_args = {
    'earth_explo_only' : False,
    'noise_earth_only' : False,
    'noise_not_noise' : True,
    'downsample' : True,
    'upsample' : True,
    'frac_diff' : 0.3,
    'seed' : 2,
    'subsample_size' : 0.3,
    'balance_non_train_set' : True,
    'use_true_test_set' : False
}
loadData = LoadData(**load_args)
full_ds, train_ds, val_ds, test_ds = loadData.get_datasets()
noise_ds = loadData.noise_ds
handler = DataHandler(loadData)

if load_args['earth_explo_only']:
    full_and_noise_ds = np.concatenate((full_ds, noise_ds))
    timeAug = TimeAugmentor(handler, full_and_noise_ds, seed = load_args['seed'])
else:
    timeAug = TimeAugmentor(handler, full_ds, seed = load_args['seed'])

In [3]:
full_ds

array([[ '/media/tord/T7/Thesis_ssd/norsar_data_nov/noise/2005-02-14T17.37.06.000000Z.h5',
        'noise', 0],
       [ '/media/tord/T7/Thesis_ssd/norsar_data_nov/noise/2017-02-13T13.55.42.000000Z.h5',
        'noise', 0],
       [ '/media/tord/T7/Thesis_ssd/norsar_data_nov/earthquakes/2016-02-11T23.43.25.442000Z.h5',
        'earthquake', 0],
       ..., 
       [ '/media/tord/T7/Thesis_ssd/norsar_data_nov/explosions/2016-06-16T12.22.17.857000Z.h5',
        'explosion', 0],
       [ '/media/tord/T7/Thesis_ssd/norsar_data_nov/explosions/2011-11-07T00.23.15.537000Z.h5',
        'explosion', 0],
       [ '/media/tord/T7/Thesis_ssd/norsar_data_nov/explosions/1996-09-08T21.00.07.474000Z.h5',
        'explosion', 0]], dtype=object)

In [6]:
############ Model picker #############
model_nr_type = "LSTM"
is_lstm = False
num_channels = 3

########### Hyperparameters ###########
batch_size = 256
epochs = 100
learning_rate = 0.01
opt = tf.keras.optimizers.Adam(learning_rate=learning_rate, clipnorm=1.0, clipvalue=0.5)
use_residuals = True
use_bottleneck = True
nr_modules = 3
kernel_size = 32
num_filters = 18
bottleneck_size = 24
shortcut_activation = "sigmoid"
output_activation = "sigmoid"
reg_module = True
reg_shortcut = True
l1_r = 0.01
l2_r = 0.01

########### Preprocessing ###########
use_noise_augmentor = True
use_time_augmentor = True
detrend = False
use_scaler = True
use_min_max = False
use_highpass = False
highpass_freq = 0.2

use_tensorboard = True
use_livelossplot = False
use_custom = False
use_reduce_lr = True


In [7]:
scaler = None
noiseAug = None
if use_time_augmentor:
    timeAug.fit()
if use_scaler:
    scaler = StandardScalerFitter(train_ds, timeAug).fit_scaler(detrend = detrend)
if use_noise_augmentor:
    noiseAug = NoiseAugmentor(train_ds, use_scaler, scaler, loadData, timeAug)

Fit process completed after 186.65945982933044 seconds. Total datapoints fitted: 33854.
Average time per datapoint: 0.005513660419133055


In [7]:
ramLoader = RamLoader(handler, timeAug, scaler)
x_train, y_train = ramLoader.load_to_ram(train_ds, is_lstm)
x_val, y_val = ramLoader.load_to_ram(val_ds, is_lstm)

Starting loading to RAM
Completed loading to RAM
Starting loading to RAM
Completed loading to RAM


In [18]:
callbacks = []

def clear_tensorboard_dir():
    import os
    import shutil
    path = f"{base_dir}/Tensorboard_dir/fit"
    files = os.listdir(path)
    print(files)
    for f in files:
        shutil.rmtree(os.path.join(path,f))

if use_tensorboard:
    import datetime
    clear_tensorboard_dir()
    %tensorboard --logdir tensorboard_dir/fit
    log_dir = f"{base_dir}/tensorboard_dir/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    callbacks.append(tensorboard_callback)

if use_custom:
    custom_callback = CustomCallback(data_gen)
    callbacks.append(custom_callback)
elif use_livelossplot:
    callbacks.append(PlotLossesKeras())
elif use_reduce_lr:
    callbacks.append(tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, patience=50,
                                                          min_lr=0.0001))

[]


In [19]:
nr_classes = len(set(y_val[:,0]))
print(nr_classes)
# Initializing model:
num_ds, channels, timesteps = handler.get_trace_shape_no_cast(train_ds, use_time_augmentor)
input_shape = (channels, timesteps)
print(input_shape)

optimizer_args = helper.generate_model_compile_args(opt, nr_classes)

build_model_args = {'input_shape' : input_shape,
                    'nr_classes' : nr_classes,
                    'optimizer_args' : optimizer_args,
                    'use_residuals' : use_residuals,
                    'use_bottleneck' : use_bottleneck,
                    'nr_modules' : nr_modules,
                    'kernel_size' : kernel_size,
                    'num_filters' : num_filters,
                    'bottleneck_size' : bottleneck_size,
                    'shortcut_activation'  : shortcut_activation,
                    'output_activation' : output_activation}

inceptionTime = InceptionTimeModel(**build_model_args)

2
(3, 6000)


In [20]:
print(optimizer_args)
nr_classes

{'loss': 'binary_crossentropy', 'optimizer': <tensorflow.python.keras.optimizer_v2.gradient_descent.SGD object at 0x7ffa400803d0>, 'metrics': [<tensorflow.python.keras.metrics.Precision object at 0x7ff8468d4d00>, <tensorflow.python.keras.metrics.BinaryAccuracy object at 0x7ffa40058820>, <tensorflow.python.keras.metrics.Recall object at 0x7ff846863160>]}


2

In [21]:
model = inceptionTime.build_model(input_shape, nr_classes)

In [22]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 3, 6000)]    0                                            
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 3, 24)        144000      input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling1d (MaxPooling1D)    (None, 3, 6000)      0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 3, 18)        13824       conv1d[0][0]                     
______________________________________________________________________________________________

In [24]:
gen = RamGenerator(loadData, handler)

train_gen = gen.data_generator(x_train, y_train, batch_size)
val_gen = gen.data_generator(x_val, y_val, batch_size)

args = {'steps_per_epoch' : helper.get_steps_per_epoch(train_ds, batch_size),
        'epochs' : epochs,
        'validation_data' : val_gen,
        'validation_steps' : helper.get_steps_per_epoch(val_ds, batch_size),
        'verbose' : 1,
        'use_multiprocessing' : False, 
        'workers' : 1,
        'callbacks' : callbacks
}

model_fit = model.fit(train_gen, **args)
#model_fit = model.fit(x_train, y_train, batch_size = batch_size, epochs = epochs, validation_data = (x_val, y_val))

Epoch 1/100
99/99 [==============================] - 6s 19ms/step - loss: 0.6838 - precision: 0.6835 - binary_accuracy: 0.5705 - recall: 0.6652 - val_loss: 0.6480 - val_precision: 0.6501 - val_binary_accuracy: 0.6491 - val_recall: 0.9975
Epoch 2/100
99/99 [==============================] - 1s 15ms/step - loss: 0.6099 - precision: 0.6622 - binary_accuracy: 0.6618 - recall: 0.9973 - val_loss: 0.6370 - val_precision: 0.6506 - val_binary_accuracy: 0.6507 - val_recall: 1.0000
Epoch 3/100
99/99 [==============================] - 2s 15ms/step - loss: 0.5942 - precision: 0.6593 - binary_accuracy: 0.6593 - recall: 0.9983 - val_loss: 0.6307 - val_precision: 0.6505 - val_binary_accuracy: 0.6505 - val_recall: 1.0000
Epoch 4/100
99/99 [==============================] - 1s 15ms/step - loss: 0.5816 - precision: 0.6604 - binary_accuracy: 0.6608 - recall: 0.9983 - val_loss: 0.6264 - val_precision: 0.6505 - val_binary_accuracy: 0.6505 - val_recall: 1.0000
Epoch 5/100
99/99 [=============================

99/99 [==============================] - 2s 16ms/step - loss: 0.0252 - precision: 0.9958 - binary_accuracy: 0.9936 - recall: 0.9946 - val_loss: 3.5459 - val_precision: 0.9741 - val_binary_accuracy: 0.5074 - val_recall: 0.2494
Epoch 70/100
99/99 [==============================] - 2s 16ms/step - loss: 0.0346 - precision: 0.9858 - binary_accuracy: 0.9873 - recall: 0.9953 - val_loss: 3.5893 - val_precision: 0.9702 - val_binary_accuracy: 0.5115 - val_recall: 0.2570
Epoch 71/100
99/99 [==============================] - 2s 15ms/step - loss: 0.0226 - precision: 0.9957 - binary_accuracy: 0.9937 - recall: 0.9947 - val_loss: 3.3283 - val_precision: 0.9695 - val_binary_accuracy: 0.5204 - val_recall: 0.2712
Epoch 72/100
99/99 [==============================] - 2s 15ms/step - loss: 0.0233 - precision: 0.9943 - binary_accuracy: 0.9937 - recall: 0.9963 - val_loss: 3.7426 - val_precision: 0.9747 - val_binary_accuracy: 0.5039 - val_recall: 0.2437
Epoch 73/100
99/99 [==============================] - 2s 

In [ ]:
y_train

In [ ]:
from Classes.ThesisConfig import ThesisConfig
